In [1]:
import json
import numpy
import pandas
from datetime import datetime
from pprint import pprint
import re
pandas.options.display.max_colwidth = -1
print('done')

done


In [2]:
# runs in about 5 minutes
#with open('events.json','r') as f:
with open('events_2012-2016.json','r') as f:
    #with open('events_1927_1938.json','r') as f:
    allgames = json.load(f)
print('ok')
#pprint(allgames[2])   
allgames_bk = allgames

allgames_df = pandas.DataFrame(allgames)
allgames_df = allgames_df.set_index('id')
print('done')

ok
done


In [3]:
#allgames_df.sample(10).index
#mygames = ['NYN201204100']
#mygames = ['BAL201208060']
mygames = ['SEA201208150']

#mygames += ['BAL201208300', 'BAL201209300', 'BAL201304210']
#mygames += ['BAL201305170', 'BAL201405010', 'BAL201406130', 'BAL201406250', 'MIA201407030', 'PIT201605180']
#mygames += ['SDN201606150', 'WAS201607200', 'BAL201608180']

#weird_game_ids = ['CLE192708190', 'BOS193008310', 'BOS193107250', 'CHA193304220', 'NYA193606030', 'MLN195505152']
#weird_game_ids += ['MIN196907170', 'NYN197208160', 'KCA201109150', 'TOR201208130', 'KCA201605310']

somecolumns = allgames_df.columns
allgames2_df = pandas.DataFrame(columns=somecolumns)
allgames2_df['id'] = ''
allgames2_df = allgames2_df.set_index('id')
allgames2_df
for thisid, thisrow in allgames_df.iterrows():
    if (thisid in mygames):
        allgames2_df = allgames2_df.append(thisrow)
allgames_df = allgames2_df
print('done')

done


In [4]:
column_list = ['event_in_game', 'event_type', 'inning', 'home_or_visitor', 'batterID']
column_list += ['pitch_count', 'pitch_sequence', 'event', 'comment', 'sub_batting_order', 'sub_position']

player_columns = ['playerID', 'name', 'batting_order', 'position']

display_order = ['gameID', 'visiting_team', 'inning', 'batting_team']
display_order += ['outs', 'balls', 'strikes', 'pitch_sequence']
display_order += ['visitor_score', 'home_score', 'batterID', 'batter_hand', 'pitcherID', 'pitcher_hand']
#display_order += ['catcherID', '1bID', '2bID', '3bID', 'ssID', 'lfID', 'cfID', 'rfID']
#display_order += ['runner1b', 'runner2b', 'runner3b']
print('ok')

ok


In [16]:
def load_rosters():
    all_rosters_df = pandas.read_csv('fullroster.csv')
    return all_rosters_df

def get_starters(onerow):
    vsdf = pandas.DataFrame(onerow['visiting_starters'], columns=player_columns)
    vsdf['which_team'] = 0
    hsdf = pandas.DataFrame(onerow['home_starters'], columns=player_columns)
    hsdf['which_team'] = 1
    df = pandas.concat((vsdf, hsdf))
    df['batting_order'] = pandas.to_numeric(df['batting_order'])
    df['position'] = pandas.to_numeric(df['position'])

    return df

def split_event(df):
    df['play'] = df['event'].str.split('/').str.get(0)
    df['modifiers'] = df['event'].str.split('/',n=1).str.get(1)
    df['play'] = df['event'].str.split('/').str.get(0)
    df['modifiers'] = df['event'].str.split('/',n=1).str.get(1)
    df['modifiers'] = df['modifiers'].fillna('')
    df['baserunning'] = df['event'].str.split('.',n=1).str.get(1)
    df['baserunning'] = df['baserunning'].fillna('')
    df['modifiers'] = df['modifiers'].str.split('.').str.get(0)
    
    return df

def get_game_info(onerow, theindex):

    info_df = pandas.DataFrame.from_records(onerow['info'], index=theindex)

    teams_df = pandas.read_csv('teams.csv')
       
    # League is "A" for AL and "N" for NL
    info_df['visleague'] = teams_df['league'][teams_df['teamID'] == info_df['visteam'].tolist()[0]].tolist()[0][0:1]
    info_df['homeleague'] = teams_df['league'][teams_df['teamID'] == info_df['hometeam'].tolist()[0]].tolist()[0][0:1]

    info_df['attendance'] = pandas.to_numeric(info_df['attendance'], errors='coerce')
    info_df['date'] = pandas.to_datetime(info_df['date'], errors='coerce')
    info_df['dayofweek'] = info_df['date'].apply(lambda x: datetime.strftime(x, '%a'))
    
    info_df['gameno'] = pandas.to_numeric(info_df['number'], errors='coerce')
    info_df = info_df.drop('number', axis=1)
    info_df['starttime'] = pandas.to_datetime(info_df['starttime'], format='%l%M%P', errors='coerce')
    info_df['temp'] = pandas.to_numeric(info_df['temp'], errors='coerce')
    info_df['timeofgame'] = pandas.to_numeric(info_df['timeofgame'], errors='coerce')
#    info_df['timeofgame'] = pandas.to_timedelta(info_df['timeofgame'].astype(str) + ' minutes', unit='m', errors='coerce')
    info_df = info_df.fillna('')

    return info_df

def get_inning_info(thisevent, infodf, inningno):
    
    thisevent['visiting_team'] = infodf['visteam'].tolist()[0]
    thisevent['home_team'] = infodf['hometeam'].tolist()[0]
    thisevent['inning'] = inningno
    if (side == 0):
        thisevent['batting_team'] = thisevent['visiting_team']
        thisevent['fielding_team'] = thisevent['home_team']
    else:
        thisevent['batting_team'] = thisevent['home_team']
        thisevent['fielding_team'] = thisevent['visiting_team']
    return thisevent

   
def get_fielders(thisevent, playing_df, fielding_flag):
                    
    thisevent['pitcherID'] = playing_df['playerID'][(playing_df['which_team'] == fielding_flag) & (playing_df['position'] == 1)].tolist()[0]                               

    thisevent['pitcher_hand'] = roster_df['throwing_hand'][
            (roster_df['playerID'] == thisevent['pitcherID']) & 
            (roster_df['year'] == float(thisevent['gameID'][3:7])) 
            ].tolist()[0]
    
    thisevent['catcherID'] = playing_df['playerID'][(playing_df['which_team'] == fielding_flag) & (playing_df['position'] == 2)].tolist()[0]
    thisevent['1bID'] = playing_df['playerID'][(playing_df['which_team'] == fielding_flag) & (playing_df['position'] == 3)].tolist()[0]
    thisevent['2bID'] = playing_df['playerID'][(playing_df['which_team'] == fielding_flag) & (playing_df['position'] == 4)].tolist()[0]
    thisevent['3bID'] = playing_df['playerID'][(playing_df['which_team'] == fielding_flag) & (playing_df['position'] == 5)].tolist()[0]
    thisevent['ssID'] = playing_df['playerID'][(playing_df['which_team'] == fielding_flag) & (playing_df['position'] == 6)].tolist()[0]
    thisevent['lfID'] = playing_df['playerID'][(playing_df['which_team'] == fielding_flag) & (playing_df['position'] == 7)].tolist()[0]
    thisevent['cfID'] = playing_df['playerID'][(playing_df['which_team'] == fielding_flag) & (playing_df['position'] == 8)].tolist()[0]
    thisevent['rfID'] = playing_df['playerID'][(playing_df['which_team'] == fielding_flag) & (playing_df['position'] == 9)].tolist()[0]

    return thisevent


def onemorebase(basey, skoa):
    if (basey[2] != ''):
        skoa = skoa + 1
        basey[2] = ''
    if (basey[1] != ''):
        basey[2] = basey[1]
    if (basey[0] != ''):
        basey[1] = basey[0]

    return basey, skoa

def advance_runners(evnt, basers, oots, scr, teem):     # basers is now a 4x2 np array

    if ((evnt['inning'] == 3) & (evnt['home_or_visitor'] == 1)):
        print('\n')

        print('Inning ', evnt['inning'],'; side ', teem)
        print('on base before: ', basers)
        print('Batter: ', evnt['batterID'])
        print('Play type: ', evnt['play'][0])
        print('baserunning info: ', evnt['baserunning'])

    # If there is baserunning info provided, obey it
    if (evnt['baserunning'] != ''):
        # If there are multiple baserunning events, split them around ;
        x = evnt['baserunning'].split(';')
        
        success = []
        failure = []
        
        for i in range(0, len(x)):
            if ('-' in x[i]):
                success.append([x[i].split('-')[0], x[i].split('-')[1][0], x[i].split('-')[1][1:]])
            if ('X' in x[i]):
                failure.append([x[i].split('X')[0], x[i].split('X')[1][0], x[i].split('X')[1][1:]])
        
        for u in (success, failure):
            for j in range(0, len(u)):
                if (u[j][0] == 'B'):
                    u[j][0] = 0
                if (u[j][1] == 'H'):
                    u[j][1] = 4
                for k in range(0,2):
                    u[j][k] = int(u[j][k])
        
        for y in range(0, len(failure)):
            oots = oots + 1
#            print('Now {0:.0f} outs'.format(oots))
#            print('failure[y]: ',failure[y])
            basers[failure[y][1]-1] = ''
#            evnt['comment'] += failure[y][2]

        for z in range(0, len(success)):
#            print('success[z]: ',success[z])
            if (success[z][1] == 4):
                scr[teem] = scr[teem] + 1
#                print('Score now {0:.0f} - {1:.0f}.'.format(scr[0], scr[1]))

            else:
                basers[success[z][1]-1] = basers[success[z][0]-1]
            basers[success[z][0]-1] = ''
            evnt['comment'] += success[z][2]

    # After baserunners have been advanced as needed, advance the batter too.
    if (evnt['play'][0] in ('W', 'I', 'F', 'E')):
#        if (evnt['baserunning'] == ''):
#            (basers, scr[teem]) = onemorebase(basers, scr[teem])
        if ('BX' not in evnt['baserunning']):   # now advance the batter to first, unless he's explicitly out
            basers[0] = evnt['batterID']

    elif (evnt['play'][0] == 'S'):
        if (evnt['play'][1] == 'B'):   # a single as opposed to a stolen base
            print('Stolen base!')
        elif ('BX' not in evnt['baserunning']):   # now advance the batter to first, unless he's explicitly out
            basers[0] = evnt['batterID']
        else:
            print('play is S but failed elifs')
        
    elif (evnt['play'][0] == 'D'):
        if ('BX' not in evnt['baserunning']):
                basers[1] = evnt['batterID']


    elif (evnt['play'][0] == 'T'):
        if ('BX' not in evnt['baserunning']):
                basers[2] = evnt['batterID']

    elif (evnt['play'][0] == 'H'):
        if (evnt['play'][1] == 'R'):
            scr[teem] = scr[teem] + 1
            basers = ['', '', '']
        elif (evnt['play'][1] == 'P'):  # HP means hit by pitch
            basers[0] = evnt['batterID']
        else:
            print('H but not R or P')
            
    else:
        print('wtf starting letter not in set of {H|S|W|I|F|E}')

    if ((evnt['inning'] == 3) & (evnt['home_or_visitor'] == 1)):
        print('on base after: ',basers)
        print('Score: ',scr)
        print('---------------')

    return ''
print('done')

done


In [17]:
roster_df = load_rosters()

events_df = pandas.DataFrame()

for thisgameID,thisrow in allgames_df.iterrows():
    print(thisgameID+'...')
    game_events_df = pandas.DataFrame()
        
    thisgame_df = pandas.DataFrame(thisrow['events'], columns=column_list)
    thisgame_df['gameID'] = thisgameID
    thisgame_df['inning'] = pandas.to_numeric(thisgame_df['inning'])
    thisgame_df['home_or_visitor'] = pandas.to_numeric(thisgame_df['home_or_visitor'])
    thisgame_df = thisgame_df.set_index('event_in_game')
    
    this_info_df = get_game_info(thisrow, allgames_df.index) # get constants of game info
    playing_df = get_starters(thisrow) # get starters
    thisgame_df = split_event(thisgame_df) # split all events in game into parts    
    score = [0,0] # score as [vis, home] starts at [0,0]

    foundinnings = thisgame_df['inning'].drop_duplicates().values.tolist()
    theseinnings = numpy.arange(1, numpy.max(foundinnings)+2) # it's +2 b/c it expects a 0-based list
     
    for ing in theseinnings:
        for side in range(0,2):                           # iter through each half-inning
            halfinn = thisgame_df[                      # get all events in this half-inning
                (thisgame_df['inning'] == ing) & 
                (thisgame_df['home_or_visitor'] == side)]
            onbase = ['', '', '']
#            onbase = numpy.empty((4,2), dtype=object)
            nOuts = 0            #     and there are no outs
            fieldingflag = 1 - side  # flag: makes fielding team the opposite of batting team

            for eig, thisevent in halfinn.iterrows():    # iter through all events in this half-inn    

                thisevent['visiting_team'] = this_info_df['visteam'].tolist()[0]
                thisevent['home_team'] = this_info_df['hometeam'].tolist()[0]
                thisevent['inning'] = ing
                if (side == 0):
                    thisevent['batting_team'] = thisevent['visiting_team']
                    thisevent['fielding_team'] = thisevent['home_team']
                else:
                    thisevent['batting_team'] = thisevent['home_team']
                    thisevent['fielding_team'] = thisevent['visiting_team']

                thisevent = get_fielders(thisevent, playing_df, fieldingflag) # get fielders, including pitcher
                
                thisevent['abflag'] = 1
                thisevent['balls'] = pandas.to_numeric(thisevent['pitch_count'][0:1], errors='coerce')
                thisevent['strikes'] = pandas.to_numeric(thisevent['pitch_count'][1:2], errors='coerce')

                # batterID gets loaded from raw data, but look up his handendess... and....
                # DON'T FORGET TO CORRECT THIS FOR BADJ IF NEEDED!
                thisevent['batter_hand'] = roster_df['batting_hand'][(roster_df['playerID'] == thisevent['batterID']) & (roster_df['team'] == thisevent['batting_team']) & (roster_df['year'] == float(thisevent['gameID'][3:7]))].tolist()[0]

                # If the play starts with fielder info or is a strikeout, increment outs
                if (re.match(r'[0-9K]+',thisevent['play'])):
                    nOuts = nOuts + 1

                # If the play is a hit, advance baserunners with fcn advance_runners(evnt, basers, oots, scr, teem):
                if (re.match(r'[S|D|T|H|W|I|E|F]',thisevent['play'])):
                    advance_runners(
                        thisevent,
                        onbase,
                        nOuts,
                        score,
                        side)
                

                # update info based on the play that has occurred
                thisevent['outs'] = nOuts
                thisevent['runner1b'] = onbase[0]
                thisevent['runner2b'] = onbase[1]
                thisevent['runner3b'] = onbase[2]
                thisevent['visitor_score'] = score[0]
                thisevent['home_score'] = score[0]

                if ((ing == 3) & (side == 1)):
                    print(eig,':',thisevent[['batterID','event']])
                    print('\n')
                # finally, append this play to the list of plays
                game_events_df = game_events_df.append(thisevent)
                
    game_events_df = game_events_df.sort_index()
    game_events_df.index.name = 'event_in_game'
    game_events_df = game_events_df.reset_index()

    events_df = pandas.concat((events_df, game_events_df), axis=0)

display_order = ['gameID', 'event_in_game', 'inning', 'batterID']
display_order += ['play', 'modifiers', 'baserunning', 'outs']
display_order += ['runner1b', 'runner2b', 'runner3b']

#events_df = events_df.fillna('')

events_df = events_df.reset_index()
events_df.index.name = 'eventID'

events_df.sort_values('gameID')
events_df = events_df[display_order]
print('Final score: ',score)
events_df
#x = events_df['play'].tolist()
#for k in x:
#    print(k)

SEA201208150...


Inning  3 ; side  1
on base before:  ['', '', '']
Batter:  ryanb002
Play type:  S
baserunning info:  
on base after:  ['ryanb002', '', '']
Score:  [0, 0]
---------------
17 : batterID    ryanb002
event       S7/L    
Name: 17, dtype: object


18 : batterID    ackld001
event       6/P     
Name: 18, dtype: object


19 : batterID    saunm001
event       7/L     
Name: 19, dtype: object




Inning  3 ; side  1
on base before:  ['ryanb002', '', '']
Batter:  montj003
Play type:  S
baserunning info:  1-3(WP)
Stolen base!
on base after:  ['', '', 'ryanb002']
Score:  [0, 0]
---------------
20 : batterID    montj003   
event       SB2.1-3(WP)
Name: 20, dtype: object




Inning  3 ; side  1
on base before:  ['', '', 'ryanb002']
Batter:  montj003
Play type:  S
baserunning info:  3-H
on base after:  ['montj003', '', '']
Score:  [0, 1]
---------------
21 : batterID    montj003
event       S7/L.3-H
Name: 21, dtype: object


22 : batterID    jasoj001
event       13/G-   
Name: 22, d

,gameID,event_in_game,inning,batterID,play,modifiers,baserunning,outs,runner1b,runner2b,runner3b
eventID,,,,,,,,,,,
0,SEA201208150,0,1.0,fulds001,9,F,,1.0,,,
1,SEA201208150,1,1.0,uptob001,63,G,,2.0,,,
2,SEA201208150,2,1.0,joycm001,43,G,,3.0,,,
3,SEA201208150,3,1.0,ackld001,S7,G,,0.0,ackld001,,
4,SEA201208150,4,1.0,saunm001,4,LDP,1X1(43),1.0,ackld001,,
5,SEA201208150,5,1.0,montj003,5,L,,2.0,ackld001,,
6,SEA201208150,6,2.0,longe001,K,,,1.0,,,
7,SEA201208150,7,2.0,zobrb001,63,G,,2.0,,,
8,SEA201208150,8,2.0,penac001,7,F,,3.0,,,
